1  ¿Cuál es la mejor tarifa?

Trabajas como analista para el operador de telecomunicaciones Megaline. La empresa ofrece a sus clientes dos tarifas de prepago, Surf y Ultimate. El departamento comercial quiere saber cuál de las tarifas genera más ingresos para poder ajustar el presupuesto de publicidad.

Vas a realizar un análisis preliminar de las tarifas basado en una selección de clientes relativamente pequeña. Tendrás los datos de 500 clientes de Megaline: quiénes son los clientes, de dónde son, qué tarifa usan, así como la cantidad de llamadas que hicieron y los mensajes de texto que enviaron en 2018. Tu trabajo es analizar el comportamiento de los clientes y determinar qué tarifa de prepago genera más ingresos.

[Te proporcionamos algunos comentarios para orientarte mientras completas este proyecto. Pero debes asegurarte de eliminar todos los comentarios entre corchetes antes de entregar tu proyecto.]

[Antes de sumergirte en el análisis de datos, explica por tu propia cuenta el propósito del proyecto y las acciones que planeas realizar.]

[Ten en cuenta que estudiar, modificar y analizar datos es un proceso iterativo. Es normal volver a los pasos anteriores y corregirlos/ampliarlos para permitir nuevos pasos.]

1.1  Inicialización

In [54]:
# Cargar todas las librerías
import pandas as pd
import numpy as np
from scipy import stats as st
import math as mt
from math import factorial


1.2  Cargar datos

In [55]:
# Carga los archivos de datos en diferentes DataFrames
calls_df = pd.read_csv('megaline_calls.csv')
internet_df = pd.read_csv('megaline_internet.csv')
messages_df = pd.read_csv('megaline_messages.csv')
plans_df = pd.read_csv('megaline_plans.csv')
users_df = pd.read_csv('megaline_users.csv')


1.3  Preparar los datos

[Los datos para este proyecto se dividen en varias tablas. Explora cada una para tener una comprensión inicial de los datos. Si es necesario, haz las correcciones requeridas en cada tabla.]

1.4  Tarifas

In [56]:
# Imprime la información general/resumida sobre el DataFrame de las tarifas
print(plans_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   messages_included      2 non-null      int64  
 1   mb_per_month_included  2 non-null      int64  
 2   minutes_included       2 non-null      int64  
 3   usd_monthly_pay        2 non-null      int64  
 4   usd_per_gb             2 non-null      int64  
 5   usd_per_message        2 non-null      float64
 6   usd_per_minute         2 non-null      float64
 7   plan_name              2 non-null      object 
dtypes: float64(2), int64(5), object(1)
memory usage: 260.0+ bytes
None


In [57]:
# Imprime una muestra de los datos para las tarifas
print('Muestra de los datos para las tarifas\n')
print(plans_df.head())

Muestra de los datos para las tarifas

   messages_included  mb_per_month_included  minutes_included  \
0                 50                  15360               500   
1               1000                  30720              3000   

   usd_monthly_pay  usd_per_gb  usd_per_message  usd_per_minute plan_name  
0               20          10             0.03            0.03      surf  
1               70           7             0.01            0.01  ultimate  


[Describe lo que ves y observas en la información general y en la muestra de datos impresa para el precio de datos anterior. ¿Hay algún problema (tipos de datos no adecuados, datos ausentes, etc.) que pudieran necesitar investigación y cambios adicionales? ¿Cómo se puede arreglar?]

El Dataset contiene 8 columnas, al parecer todas tiene los tipos correctos de datos y los nombres son claros a excepcion de usd_monthly_pay, creo que seria mejor usd_montly_fee, ninguna columna tiene datos ausentes. solo es actualizar el nombre de 1 columna.

1.5  Corregir datos

[Corrige los problemas obvios con los datos basándote en las observaciones iniciales.]

In [58]:
# Cambio de nombre en columna para mejor claridad y descripcion
plans_df = plans_df.rename(columns = {'mb_per_month_included': 'mb_included',
                                      'usd_monthly_pay': 'usd_monthly_fee',
                                      'usd_per_gb': 'usd_per_gb_exceeded',
                                      'usd_per_message': 'usd_per_message_exceeded',
                                      'usd_per_minute': 'usd_per_minute_exceeded'}
                          )
print(plans_df.columns)

Index(['messages_included', 'mb_included', 'minutes_included',
       'usd_monthly_fee', 'usd_per_gb_exceeded', 'usd_per_message_exceeded',
       'usd_per_minute_exceeded', 'plan_name'],
      dtype='object')


1.6  Enriquecer los datos

[Agrega factores adicionales a los datos si crees que pudieran ser útiles.]

In [59]:
# Agrega factores adicionales a los datos para mayor profundidad en el analisis
plans_df['plan_type'] = plans_df['usd_monthly_fee'].apply(lambda x: 'Premium' if x > 20 else 'Basic')
plans_df['usd_per_minute_cost'] = plans_df['usd_monthly_fee'] / plans_df['minutes_included']
plans_df['usd_per_gb_cost'] = plans_df['usd_monthly_fee'] / (plans_df['mb_included'] / 1240)
plans_df['usd_per_sms_cost'] = plans_df['usd_monthly_fee'] / plans_df['messages_included']

print('\nCosto unitario por servicio incluido en cada plan\n')
print(plans_df)


Costo unitario por servicio incluido en cada plan

   messages_included  mb_included  minutes_included  usd_monthly_fee  \
0                 50        15360               500               20   
1               1000        30720              3000               70   

   usd_per_gb_exceeded  usd_per_message_exceeded  usd_per_minute_exceeded  \
0                   10                      0.03                     0.03   
1                    7                      0.01                     0.01   

  plan_name plan_type  usd_per_minute_cost  usd_per_gb_cost  usd_per_sms_cost  
0      surf     Basic             0.040000         1.614583              0.40  
1  ultimate   Premium             0.023333         2.825521              0.07  


1.7  Usuarios/as

In [60]:
# Imprime la información general/resumida sobre el DataFrame de usuarios
print('Informacion general del dataset Usuarios\n')
print(users_df.info())

Informacion general del dataset Usuarios

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     500 non-null    int64 
 1   first_name  500 non-null    object
 2   last_name   500 non-null    object
 3   age         500 non-null    int64 
 4   city        500 non-null    object
 5   reg_date    500 non-null    object
 6   plan        500 non-null    object
 7   churn_date  34 non-null     object
dtypes: int64(2), object(6)
memory usage: 31.4+ KB
None


In [61]:
# Imprime una muestra de datos para usuarios
print('Vista general de dataset Usuarios\n')
print(users_df.head())
print('\nMuestra de datos para dataset Usuarios\n')
print(users_df.sample(10))

Vista general de dataset Usuarios

   user_id first_name  last_name  age                                   city  \
0     1000   Anamaria      Bauer   45  Atlanta-Sandy Springs-Roswell, GA MSA   
1     1001     Mickey  Wilkerson   28        Seattle-Tacoma-Bellevue, WA MSA   
2     1002     Carlee    Hoffman   36   Las Vegas-Henderson-Paradise, NV MSA   
3     1003   Reynaldo    Jenkins   52                          Tulsa, OK MSA   
4     1004    Leonila   Thompson   40        Seattle-Tacoma-Bellevue, WA MSA   

     reg_date      plan churn_date  
0  2018-12-24  ultimate        NaN  
1  2018-08-13      surf        NaN  
2  2018-10-21      surf        NaN  
3  2018-01-28      surf        NaN  
4  2018-05-23      surf        NaN  

Muestra de datos para dataset Usuarios

     user_id first_name  last_name  age  \
46      1046      Beata     Hooper   67   
427     1427      Zofia      Brock   64   
354     1354     Leonel       Dyer   41   
41      1041   Drucilla       Lynn   58   
316   

[Describe lo que ves y observas en la información general y en la muestra de datos impresa para el precio de datos anterior. ¿Hay algún problema (tipos de datos no adecuados, datos ausentes, etc.) que pudieran necesitar investigación y cambios adicionales? ¿Cómo se puede arreglar?]

Dentro del dataset se pueden hacer algunas correcciones en los nombres para que sean mas claros, tipo de datos y rellenar los valores nulos. Tambien note que el nombre de la cuidad de residencia del usuario tiene multiples ciudades, entonces podriamos filtrar los datos por la primera cuidad solamente para hacer un analisis mas correcto en el futuro.

1.7.1  Corregir los datos

[Corrige los problemas obvios con los datos basándote en las observaciones iniciales.]

In [62]:
# Modificacion de nombre en columna
users_df = users_df.rename(columns={'reg_date': 'registration_date'})

# Modificacion de datatype en columnas
users_df['registration_date'] = pd.to_datetime(users_df['registration_date'])
users_df['churn_date'] = pd.to_datetime(users_df['churn_date'], errors='coerce')

# Verify changes on dataset
print(users_df.head())
print()
print(users_df.info())

   user_id first_name  last_name  age                                   city  \
0     1000   Anamaria      Bauer   45  Atlanta-Sandy Springs-Roswell, GA MSA   
1     1001     Mickey  Wilkerson   28        Seattle-Tacoma-Bellevue, WA MSA   
2     1002     Carlee    Hoffman   36   Las Vegas-Henderson-Paradise, NV MSA   
3     1003   Reynaldo    Jenkins   52                          Tulsa, OK MSA   
4     1004    Leonila   Thompson   40        Seattle-Tacoma-Bellevue, WA MSA   

  registration_date      plan churn_date  
0        2018-12-24  ultimate        NaT  
1        2018-08-13      surf        NaT  
2        2018-10-21      surf        NaT  
3        2018-01-28      surf        NaT  
4        2018-05-23      surf        NaT  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   user_id            500 non-null    int64 

1.7.2  Enriquecer los datos

[Agrega factores adicionales a los datos si crees que pudieran ser útiles.]

In [63]:
#Obtener la duracion de suscripcion por cliente
users_df['subscription_duration'] = (users_df['churn_date'] - users_df['registration_date']).dt.days
print('Duracion de suscripcion por cliente\n')
print(users_df['subscription_duration'])

# Verificar datos duplicados
duplicates = users_df[users_df.duplicated(subset=['user_id'], keep=False)]
print('usuarios_duplicados: ', duplicates)

Duracion de suscripcion por cliente

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
495   NaN
496   NaN
497   NaN
498   NaN
499   NaN
Name: subscription_duration, Length: 500, dtype: float64
usuarios_duplicados:  Empty DataFrame
Columns: [user_id, first_name, last_name, age, city, registration_date, plan, churn_date, subscription_duration]
Index: []


1.8  Llamadas

In [64]:
# Imprime la información general/resumida sobre el DataFrame de las llamadas
print(calls_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137735 entries, 0 to 137734
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id         137735 non-null  object 
 1   user_id    137735 non-null  int64  
 2   call_date  137735 non-null  object 
 3   duration   137735 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.2+ MB
None


In [65]:
# Imprime una muestra de datos para las llamadas
print(calls_df.sample(10))

              id  user_id   call_date  duration
127975  1459_119     1459  2018-12-25      9.45
36149   1138_102     1138  2018-09-08      0.00
112043  1393_247     1393  2018-11-02      9.02
62471   1224_116     1224  2018-08-08     12.13
9731    1043_291     1043  2018-07-17     17.76
85460   1313_440     1313  2018-12-09      0.53
81042   1291_642     1291  2018-12-17      5.12
65135   1236_424     1236  2018-09-19     12.72
68819   1247_152     1247  2018-09-10      1.29
53775   1193_216     1193  2018-12-02      8.64


[Describe lo que ves y observas en la información general y en la muestra de datos impresa para el precio de datos anterior. ¿Hay algún problema (tipos de datos no adecuados, datos ausentes, etc.) que pudieran necesitar investigación y cambios adicionales? ¿Cómo se puede arreglar?]

Creo que en este dataset podriamos hacer algunos cambios en el nobmre de las columnas 'id' and 'duration' para que sean mas homogeneos los nombres, tambien cambiar el datatype de 'call_date' por datetime64,

1.8.1  Corregir los datos

[Corrige los problemas obvios con los datos basándote en las observaciones iniciales.]

In [66]:
# Modify calls_df columns name
calls_df = calls_df.rename(columns={'id': 'caller_id', 'duration': 'call_duration'})

# Modify datatype for column 'call_date'
calls_df['call_date'] = pd.to_datetime(calls_df['call_date'])

# Verify changes on dataset
print(calls_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137735 entries, 0 to 137734
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   caller_id      137735 non-null  object        
 1   user_id        137735 non-null  int64         
 2   call_date      137735 non-null  datetime64[ns]
 3   call_duration  137735 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 4.2+ MB
None


1.8.2  Enriquecer los datos

[Agrega factores adicionales a los datos si crees que pudieran ser útiles.]

In [67]:
# Analizar llamadas por usuario
calls_per_user = calls_df.groupby('user_id')['call_date'].count().reset_index()
print('Llamadas por usuario\n', calls_per_user)

# Cual es el maximo tiempo en duracion de llamadas
max_call_duration = calls_df['call_duration'].max()
print(f'\nDuracion maxima de una llamda: {max_call_duration} minutos\n')

# lapso de tiempo entre llamadas
# Ordenar por usuario y fecha de llamada
calls_df = calls_df.sort_values(by=['user_id', 'call_date'])
# calcular tiempo entre llamadas
calls_df['time_between_calls'] = calls_df.groupby('user_id')['call_date'].diff()
# Imprimir resultado
print('\nLapso de tiempo entre llamadas por usuario\n')
print(calls_df[['user_id', 'call_date', 'time_between_calls']])


Llamadas por usuario
      user_id  call_date
0       1000         16
1       1001        261
2       1002        113
3       1003        149
4       1004        370
..       ...        ...
476     1495        253
477     1496        195
478     1497         54
479     1498        451
480     1499        204

[481 rows x 2 columns]

Duracion maxima de una llamda: 37.6 minutos


Lapso de tiempo entre llamadas por usuario

        user_id  call_date time_between_calls
9          1000 2018-12-26                NaT
14         1000 2018-12-26             0 days
0          1000 2018-12-27             1 days
1          1000 2018-12-27             0 days
2          1000 2018-12-27             0 days
...         ...        ...                ...
137708     1499 2018-12-29             1 days
137734     1499 2018-12-29             0 days
137638     1499 2018-12-30             1 days
137587     1499 2018-12-31             1 days
137709     1499 2018-12-31             0 days

[137735 rows x 3 colum

1.9  Mensajes

In [68]:
# Imprime la información general/resumida sobre el DataFrame de los mensajes
print(messages_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76051 entries, 0 to 76050
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            76051 non-null  object
 1   user_id       76051 non-null  int64 
 2   message_date  76051 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB
None


In [69]:
# Imprime una muestra de datos para los mensajes
print(messages_df.sample(10))


             id  user_id message_date
58252   1369_53     1369   2018-12-02
59613   1379_90     1379   2018-12-21
16236  1110_127     1110   2018-07-02
50844   1331_19     1331   2018-09-25
37451   1248_15     1248   2018-12-09
26303  1159_306     1159   2018-09-15
55699    1351_3     1351   2018-12-30
22745  1136_200     1136   2018-10-19
11371  1077_642     1077   2018-12-19
67545  1438_111     1438   2018-09-23


[Describe lo que ves y observas en la información general y en la muestra de datos impresa para el precio de datos anterior. ¿Hay algún problema (tipos de datos no adecuados, datos ausentes, etc.) que pudieran necesitar investigación y cambios adicionales? ¿Cómo se puede arreglar?] 

El data set contiene informacion sobre el servicio de SMS, fecha e identificadores de usario y mensajes, hace falta un nombre mas claro para la columna 'id' y tambien se necesita corregir el datatype de 'message_date'

1.9.1  Corregir los datos

[Corrige los problemas obvios con los datos basándote en las observaciones iniciales.]

In [70]:
# Modificar el nombre de la columna 'id'
messages_df = messages_df.rename(columns={'id': 'message_id'})

# Corregir el datatype de 'message_date'
messages_df['message_date'] = pd.to_datetime(messages_df['message_date'])

print(messages_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76051 entries, 0 to 76050
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   message_id    76051 non-null  object        
 1   user_id       76051 non-null  int64         
 2   message_date  76051 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 1.7+ MB
None


1.9.2  Enriquecer los datos

[Agrega factores adicionales a los datos si crees que pudieran ser útiles.]

In [71]:
# Calcula mensajes por usuario
messages_per_user = messages_df.groupby('user_id')['message_id'].count().reset_index()
messages_per_user = messages_per_user.rename(columns={'message_id': 'total_messages'})

# Mostrar los mensajes usados por plan, combina users_df con messages_df
# Combinar mensajes por usuario
merged_df = messages_per_user.merge(users_df, on = 'user_id', how='left')

# Combinar planes
merged_df =  merged_df.merge(plans_df, left_on='plan', right_on='plan_name', how = 'left')
#print(merged_df)

merged_df['extra_messages'] = merged_df['total_messages'] - merged_df['messages_included']
merged_df['extra_messages'] = merged_df['extra_messages'].apply(lambda x: max(0,x))

# Imprimir resultado
print(merged_df[['user_id', 'total_messages', 'messages_included', 'extra_messages']].head())


   user_id  total_messages  messages_included  extra_messages
0     1000              11               1000               0
1     1001             207                 50             157
2     1002              88                 50              38
3     1003              50                 50               0
4     1004             177                 50             127


1.10  Internet

In [72]:
# Imprime la información general/resumida sobre el DataFrame de internet
print(internet_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104825 entries, 0 to 104824
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            104825 non-null  object 
 1   user_id       104825 non-null  int64  
 2   session_date  104825 non-null  object 
 3   mb_used       104825 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 3.2+ MB
None


In [73]:
# Imprime una muestra de datos para el tráfico de internet
print(internet_df.sample(10))


             id  user_id session_date  mb_used
79730    1373_9     1373   2018-09-19   480.90
13559    1064_7     1064   2018-07-21   277.59
31855  1144_611     1144   2018-06-23   487.75
13606   1064_54     1064   2018-09-10   539.11
17948    1081_3     1081   2018-05-29     0.00
23141  1106_445     1106   2018-06-24     6.81
71980  1335_220     1335   2018-07-04   615.06
13938   1065_34     1065   2018-10-05    20.39
22858   1106_36     1106   2018-11-19   251.86
75425   1355_35     1355   2018-11-20   624.37


[Describe lo que ves y observas en la información general y en la muestra de datos impresa para el precio de datos anterior. ¿Hay algún problema (tipos de datos no adecuados, datos ausentes, etc.) que pudieran necesitar investigación y cambios adicionales? ¿Cómo se puede arreglar?]

Este Dataset tiene 4 columnas que continen la informacion del uso del servicio de internet, se necesitan hacer cambios en el nombre del ID de sesion por un nombre mas descriptivo y hacer el cambio de datatype en la columna 'session_date" a tipo de datos de fechas, tiene algunos datos en 0 para la columna de 'mb_used', el cual seria bueno investigar cuantos usarios no usan megas, no contiene datos nulos asi que no es necesario hacer ningun otro ajuste a la tabla.

1.10.1  Corregir los datos

[Corrige los problemas obvios con los datos basándote en las observaciones iniciales.]

In [74]:
# Modifica el datatype de 'session_date'
internet_df['session_date'] = pd.to_datetime(internet_df['session_date'])

# Modifica el nombre de la columna 'id' se sesion
internet_df = internet_df.rename(columns={'id': 'session_id'})

# Imprime los datos generales con los cambios
print(internet_df.info())
print()
print(internet_df.sample(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104825 entries, 0 to 104824
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   session_id    104825 non-null  object        
 1   user_id       104825 non-null  int64         
 2   session_date  104825 non-null  datetime64[ns]
 3   mb_used       104825 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 3.2+ MB
None

      session_id  user_id session_date  mb_used
56297   1254_299     1254   2018-08-14     0.00
32713   1147_347     1147   2018-08-14    92.95
74498    1350_72     1350   2018-12-16   156.23
64388   1295_360     1295   2018-10-03   147.35
18183   1081_251     1081   2018-08-03   304.67
19742    1089_22     1089   2018-12-29     0.00
31442   1144_157     1144   2018-07-20   588.99
33366   1151_108     1151   2018-11-17   318.82
71189   1332_162     1332   2018-12-07   290.24
58094   1261_571 

1.10.2  Enriquecer los datos

[Agrega factores adicionales a los datos si crees que pudieran ser útiles.]

In [75]:
# Combinar internet_df con users_df
internet_merge_df = internet_df.merge(users_df, on='user_id', how='left')

# Combinar internet_merge_df con plans_df
internet_merge_df = internet_merge_df.merge(plans_df, left_on='plan', right_on='plan_name', how='left')

# Muestra Megabites usados por usario basado en el plan
total_mb_per_user = internet_merge_df.groupby('user_id')['mb_used'].sum().reset_index()
total_mb_per_user = total_mb_per_user.rename(columns={'mb_used': 'total_mb_used'})

internet_merge_df = internet_merge_df.merge(total_mb_per_user, on='user_id', how='left') 
print(internet_merge_df.columns)

# Muestra usarios que compran Megabites extras
internet_merge_df['extra_mb_used'] = internet_merge_df['total_mb_used'] - internet_merge_df['mb_included']
internet_merge_df['extra_mb_used'] = internet_merge_df['extra_mb_used'].apply(lambda x: max(0,x))

# Crear columna para extra megas comprados
internet_merge_df['bought_extra_mb'] = internet_merge_df['extra_mb_used'] > 0
print()
print(internet_merge_df[['total_mb_used', 'plan_type', 'plan_name', 'bought_extra_mb', 'extra_mb_used']])

# Megabites usados segun la edad del usuario
internet_merge_df['age_group'] = pd.cut(
    internet_merge_df['age'],
    bins=[10,18,25,35,45,55,65,100],
    labels=['0-18', '19-25', '26-35', '36-45', '46-55', '56-65', '65+']
    )
# Calculo de el uso promedio de datos por edad
mb_usage_by_age = internet_merge_df.groupby('age_group')['total_mb_used'].mean().reset_index()
mb_usage_by_age = mb_usage_by_age.rename(columns={'total_mb_used': 'average_mb_used'})

print('\nMegabites usados segun la edad de los usuarios\n')
print(mb_usage_by_age.head(7))

# Usuarios que no usan Megabites en ninguna sesion
users_with_zero_mb = internet_merge_df[internet_merge_df['total_mb_used'] == 0.0]['user_id'].nunique()
print(f'\nNumero de usuarios que no han usado datos: {users_with_zero_mb}')

# Usuarios que tienen sesiones individuales sin uso de datos
zero_mb_sessions = internet_df[internet_df['mb_used']==0].shape[0]
print(f'\nNumero de sessiones con 0 MB usados: {zero_mb_sessions}')


Index(['session_id', 'user_id', 'session_date', 'mb_used', 'first_name',
       'last_name', 'age', 'city', 'registration_date', 'plan', 'churn_date',
       'subscription_duration', 'messages_included', 'mb_included',
       'minutes_included', 'usd_monthly_fee', 'usd_per_gb_exceeded',
       'usd_per_message_exceeded', 'usd_per_minute_exceeded', 'plan_name',
       'plan_type', 'usd_per_minute_cost', 'usd_per_gb_cost',
       'usd_per_sms_cost', 'total_mb_used'],
      dtype='object')

        total_mb_used plan_type plan_name  bought_extra_mb  extra_mb_used
0             1901.47   Premium  ultimate            False           0.00
1             1901.47   Premium  ultimate            False           0.00
2             1901.47   Premium  ultimate            False           0.00
3             1901.47   Premium  ultimate            False           0.00
4             1901.47   Premium  ultimate            False           0.00
...               ...       ...       ...              ...     

C:\Users\jovan\AppData\Local\Temp\ipykernel_12684\3373741274.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mb_usage_by_age = internet_merge_df.groupby('age_group')['total_mb_used'].mean().reset_index()


1.11  Estudiar las condiciones de las tarifas

[Es sumamente importante entender cómo funcionan las tarifas, cómo se les cobra a los usuarios en función de su plan de suscripción. Así que te sugerimos imprimir la información de la tarifa para ver una vez más sus condiciones.]

In [76]:
# Imprime las condiciones de la tarifa y asegúrate de que te quedan claras
print(plans_df.head())
      

   messages_included  mb_included  minutes_included  usd_monthly_fee  \
0                 50        15360               500               20   
1               1000        30720              3000               70   

   usd_per_gb_exceeded  usd_per_message_exceeded  usd_per_minute_exceeded  \
0                   10                      0.03                     0.03   
1                    7                      0.01                     0.01   

  plan_name plan_type  usd_per_minute_cost  usd_per_gb_cost  usd_per_sms_cost  
0      surf     Basic             0.040000         1.614583              0.40  
1  ultimate   Premium             0.023333         2.825521              0.07  


1.12  Agregar datos por usuario

[Ahora que los datos están limpios, agrega los datos por usuario y por periodo para que solo haya un registro por usuario y por periodo. Esto facilitará mucho el análisis posterior.]

In [77]:
# Calcula el número de llamadas hechas por cada usuario al mes. Guarda el resultado.
#calls_per_user = calls_df.groupby('user_id')['call_date'].count().reset_index()
print('Llamadas por usario\n')
print(calls_per_user) # Variable para este calculo fue hecho en la seccion 'Enriquecer datos' de la seccion Calls

Llamadas por usario

     user_id  call_date
0       1000         16
1       1001        261
2       1002        113
3       1003        149
4       1004        370
..       ...        ...
476     1495        253
477     1496        195
478     1497         54
479     1498        451
480     1499        204

[481 rows x 2 columns]


In [78]:
# Calcula la cantidad de minutos usados por cada usuario al mes. Guarda el resultado.
minutes_call_per_user = calls_df.groupby('user_id')['call_duration'].count().reset_index()
print(minutes_call_per_user)

     user_id  call_duration
0       1000             16
1       1001            261
2       1002            113
3       1003            149
4       1004            370
..       ...            ...
476     1495            253
477     1496            195
478     1497             54
479     1498            451
480     1499            204

[481 rows x 2 columns]


In [79]:
# Calcula el número de mensajes enviados por cada usuario al mes. Guarda el resultado.
#messages_per_user = messages_df.groupby('user_id')['message_id'].count().reset_index()
print(messages_per_user) # Variable para este calculo fue hecho en la seccion 'Enriquecer datos' de la seccion Messages

     user_id  total_messages
0       1000              11
1       1001             207
2       1002              88
3       1003              50
4       1004             177
..       ...             ...
397     1491             409
398     1492             108
399     1494             174
400     1496              65
401     1497              50

[402 rows x 2 columns]


In [80]:
# Calcula el volumen del tráfico de Internet usado por cada usuario al mes. Guarda el resultado.
mb_per_user = internet_df.groupby('user_id')['mb_used'].count().reset_index()
print(mb_per_user)

     user_id  mb_used
0       1000        5
1       1001      245
2       1002      124
3       1003       52
4       1004      460
..       ...      ...
484     1495      290
485     1496      225
486     1497       31
487     1498      657
488     1499      220

[489 rows x 2 columns]
